# IAI CNC Competiton 2018

Copyright © 2019 Hsu Shih-Chieh

刀具铣切次数預測

In [3]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
import sys, os, datetime, warnings, scipy, collections, random, pywt, time, matplotlib
#traceback, shutil, progressbar, pylab, itertools, pickle
import pandas as pd 
import numpy as np
from scipy import signal, stats
from scipy.stats import skew, kurtosis, norm
from sklearn.externals import joblib 
from sklearn import decomposition, mixture
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
from datetime import timedelta
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import skew, kurtosis, norm
from sklearn.base import BaseEstimator, RegressorMixin
from datasets import load_iaicnc
from utils import set_font_cn
from utils import Bunch
from utils import cnc_featuring, cnc_predict


set_font_cn()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Data

In [4]:
datasets = load_iaicnc()
print(data.DESCR)

2it [00:00, 11.54it/s]

重新讀取數據
wear_A (315, 6)
wear_B (273, 6)
/Users/jayhsu/work/Intelligent Manufacturing/datasets/data/iai_cnc/Train_A/Train_A 315


315it [00:36,  8.61it/s]
0it [00:00, ?it/s]

/Users/jayhsu/work/Intelligent Manufacturing/datasets/data/iai_cnc/Train_B/Train_B 315


315it [00:30, 10.41it/s]
1it [00:00,  9.40it/s]

/Users/jayhsu/work/Intelligent Manufacturing/datasets/data/iai_cnc/Test/Test 315


315it [00:35,  8.85it/s]


NameError: name 'data' is not defined

### Featuring

In [ ]:
if os.path.exists('model/iaicnc_feature.pkl'):
    print('讀取特徵暫存檔')
    cache_feature = joblib.load('model/iaicnc_feature.pkl')   
    trA_feature=cache_feature.trA_feature
    trB_feature=cache_feature.trB_feature
    ts_feature = cache_feature.ts_feature
else:
    print('暫存檔不存在, 重新計算特徵')
    trA_feature = cnc_featuring(datasets.trA)
    trB_feature = cnc_featuring(datasets.trB)
    ts_feature = cnc_featuring(datasets.ts)   
    cache_feature = Bunch(trA_feature=trA_feature, trB_feature=trB_feature, ts_feature=ts_feature, names = trA_feature.columns)
    joblib.dump(cache_feature,'model/iaicnc_feature.pkl')       


### Pre-Process
- 刪掉不要的feature
- 兩份刀具的數據合併

In [ ]:
selectedfeature = trA_feature.columns
#刪掉不要的feature
if True:
    selectedfeature = list(filter(lambda x: 'v_stableFreq' not in x, selectedfeature)) #v取樣頻率太小, 沒辦法使用頻域特徵
    selectedfeature = list(filter(lambda x: 'g_cut1mean' not in x, selectedfeature)) #震動的cut-in, cut-out mean看起來沒有趨勢
    selectedfeature = list(filter(lambda x: 'g_cut2mean' not in x, selectedfeature))
print('feature qty:',len(selectedfeature))



#Combind AX, BX, Ay, By
data_ABX = pd.concat((trA_feature[selectedfeature], trB_feature[selectedfeature]))
data_ABy_min = np.concatenate((datasets.wear_A['flute_min'], datasets.wear_B['flute_min']),axis = 0)
data_ABy_mean = np.concatenate((datasets.wear_A['flute_mean'], datasets.wear_B['flute_mean']),axis = 0)
data_ABy_max = np.concatenate((datasets.wear_A['flute_max'], datasets.wear_B['flute_max']),axis = 0)
tsx = ts_feature[selectedfeature]

def getTrainingData(y):
    X_tr, X_ts, Y_tr, Y_ts, _, _ = train_test_split(data_ABX, y, np.arange(len(y.flatten())), random_state=0, test_size=1)
    X_tr = X_tr.fillna(method='ffill')
    X_tr = X_tr.fillna(method='bfill')
    X_ts = X_ts.fillna(method='ffill')
    X_ts = X_ts.fillna(method='bfill')
    return X_tr, X_ts, Y_tr, Y_ts

X_tr, X_ts, Y_tr, Y_ts = getTrainingData(data_ABy_min)
print('X_tr shape:',X_tr.shape)
print('X_ts shape:',X_ts.shape)


### Model Training

In [ ]:
starttime = datetime.datetime.now()
#---- 分別用磨耗mean與max建立Model ----
xgbr = xgb.XGBRegressor(n_jobs=-1, n_estimators=300, max_depth=5, reg_lambda=0.9,  learning_rate=0.03, subsample=0.9, colsample_bylevel=0.9, colsample_bytree=0.9) 
X_tr, X_ts, Y_tr, Y_ts = getTrainingData(y=data_ABy_mean)
model_mean = xgbr.fit(X_tr.values, Y_tr)

xgbr = xgb.XGBRegressor(n_jobs=-1, n_estimators=200, max_depth=5, reg_lambda=0.9,  learning_rate=0.03, subsample=0.9, colsample_bylevel=0.9, colsample_bytree=0.9) 
X_tr, X_ts, Y_tr, Y_ts = getTrainingData(y=data_ABy_max)
model_max = xgbr.fit(X_tr.values, Y_tr)
walltime = (datetime.datetime.now() - starttime)
print('Model Training', 'Wall time:', round(walltime.total_seconds(),2), 'secondes')


### Prediction

In [ ]:
#%%time
starttime = datetime.datetime.now()
cutcnt_max = cnc_predict(model_mean, model_max)
walltime = (datetime.datetime.now() - starttime)
print('Prediction','Wall time:', round(walltime.total_seconds(),3), 'secondes')
